<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/LLama_prompt/03_Sentiment_Analysis/LLama_prompt_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing libraries**


In [1]:
!pip install -q -U langchain langchain_community transformers bitsandbytes accelerate

**Importing libraries**

In [2]:
import os
import pandas as pd

import torch
from transformers import BitsAndBytesConfig
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from huggingface_hub import HfFolder, HfApi
from google.colab import userdata


In [3]:
# Retrieve the token from Colab secrets
token = userdata.get('HF_TOKEN')
if token:
    HfFolder.save_token(token)
    api = HfApi()
    user_info = api.whoami()
    print(user_info)

    # Print success message
    if user_info:
        print("Connection to Hugging Face was successful.")
    else:
        print("Failed to connect to Hugging Face. Please check your token.")
else:
    print("Hugging Face token not found. Please set the HF_TOKEN environment variable.")

{'type': 'user', 'id': '6653b672ccdee6254464ad77', 'name': 'IK25', 'fullname': 'Kussanov', 'email': 'ilyas.kussanov@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/1cfaf9904ec6d92a21589601454a442a.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Capstone', 'role': 'read', 'createdAt': '2024-05-27T01:19:50.573Z'}}}
Connection to Hugging Face was successful.


In [4]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone --branch LLama_prompt https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

fatal: destination path 'Financial_Sentiment_LLM' already exists and is not an empty directory.


In [5]:


%cd Financial_Sentiment_LLM/03_Sentiment_Analysis/source_code



[Errno 2] No such file or directory: 'Financial_Sentiment_LLM/03_Sentiment_Analysis/source_code'
/content


In [ ]:
# Import the 'utilis' module
from utilis import initialize_model, initialize_llm, process_article

In [ ]:
initialize_model()
initialize_llm()

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
invest_df1 = pd.read_csv('https://raw.githubusercontent.com/Kussil/Financial_Sentiment_LLM/main/02_Cleaned_Data/Investment_Research_Part1.csv?token=GHSAT0AAAAAACS4ZNKAIQEK5VO57IDSOPQ6ZTRBTTA')
# invest_df2 = pd.read_csv('https://raw.githubusercontent.com/Kussil/Financial_Sentiment_LLM/main/02_Cleaned_Data/Investment_Research_Part1.csv?token=GHSAT0AAAAAACS4ZNKAIQEK5VO57IDSOPQ6ZTRBTTA')
# proquest_df = pd.read_csv('https://raw.githubusercontent.com/Kussil/Financial_Sentiment_LLM/main/02_Cleaned_Data/ProQuest_Articles.csv?token=GHSAT0AAAAAACS4ZNKBDO3563MRUPSD2DPUZTQ7K5Q')
# earnings_presentations = pd.read_csv('https://raw.githubusercontent.com/Kussil/Financial_Sentiment_LLM/main/02_Cleaned_Data/Earnings_Presentations.csv?token=GHSAT0AAAAAACS4ZNKAWHJZXEW4UAHSJ2ASZTQ7LYQ')
# earnings_qa = pd.read_csv('https://raw.githubusercontent.com/Kussil/Financial_Sentiment_LLM/main/02_Cleaned_Data/Earnings_QA.csv?token=GHSAT0AAAAAACS4ZNKBMDHR46MINJE2FRWMZTQ7MFQ')
# sec_df = pd.read_csv('https://raw.githubusercontent.com/Kussil/Financial_Sentiment_LLM/main/02_Cleaned_Data/SEC_Filings.csv?token=GHSAT0AAAAAACS4ZNKBV5KI4YJS3IC5U6CEZTQ7MPQ')


In [ ]:
# Merge into single df
#text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
text_df = invest_df1
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(2440, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
2435,Investment Research,IR-2448,HES,2024-01-31,Hess Corporation,"Stock Report Front| January 31, 2024 | NYSESym...",NaN
2436,Investment Research,IR-2449,PSX,2024-01-31,Phillips 66,"Stock Report Front| January 31, 2024 | NYSESym...",NaN
2437,Investment Research,IR-2450,MPC,2024-01-30,Marathon Petroleum Corporation,"Stock Report Front| January 30, 2024 | NYSESym...",NaN
2438,Investment Research,IR-2451,VLO,2024-01-30,Reaffirming BUY following 4Q23 results,NNNNYYYYSSSSEEEE:::: VVVVLLLLOOOO\nVVVVAAAALLL...,NaN
2439,Investment Research,IR-2452,VLO,2024-01-29,Valero Energy Corporation,"Stock Report Front| January 29, 2024 | NYSESym...",NaN


In [ ]:
print(text_df.head(2))

                Source Unique_ID Ticker        Date          Article Headline  \
0  Investment Research      IR-1    MRO  2024-05-16  Marathon Oil Corporation   
1  Investment Research      IR-2    EOG  2024-05-14       EOG Resources, Inc.   

                                        Article Text  URL  
0  Stock Report | May 16, 2024 | NYSESymbol: MRO ...  NaN  
1  Stock Report | May 14, 2024 | NYSESymbol: EOG ...  NaN  


In [ ]:
# Drop rows google gemini will not process
rows_to_drop = ['PQ-2840736837']
index_to_drops = text_df[text_df['Unique_ID'].isin(rows_to_drop)].index
text_df.drop(index_to_drops, inplace=True)
print(index_to_drops)

Index([], dtype='int64')


In [ ]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"]
SENTIMENT_RESULTS_FILE_PATH = 'LLAMA_Sentiment_Analysis_Results.csv'

In [ ]:
import os

# Print the current working directory
print(f"Current working directory: {os.getcwd()}")

# Determine if the Sentiment Analysis Results file already exists
SENTIMENT_RESULTS_FILE_PATH = 'path_to_your_file'  # Replace with your actual file path
file_exists = os.path.isfile(SENTIMENT_RESULTS_FILE_PATH)

# Print the result
if file_exists:
    print(f"The file exists in the current directory.")
else:
    print(f"The file does not exist in the current directory.")


Current working directory: /content/Financial_Sentiment_LLM/03_Sentiment_Analysis/source_code
The file does not exist in the current directory.


In [ ]:
# Create an empty file if the file does not exist
if not file_exists:
    # Copy text df and drop the text and headline column due to size
    empty_sentiment_df = text_df.copy()
    empty_sentiment_df = empty_sentiment_df.drop(['Article Text', 'Article Headline'], axis=1)

    for category in CATEGORIES:
        empty_sentiment_df[category] = ""
        empty_sentiment_df[category] = empty_sentiment_df[category].astype('object')

    # Display results
    display(empty_sentiment_df.head())

    # Save as CSV
    empty_sentiment_df.to_csv(SENTIMENT_RESULTS_FILE_PATH, index=False)

,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves / Exploration / Acquisitions / Mergers / Divestments,Environment / Regulatory / Geopolitics,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price
0,Investment Research,IR-1,MRO,2024-05-16,NaN,,,,,,
1,Investment Research,IR-2,EOG,2024-05-14,NaN,,,,,,
2,Investment Research,IR-3,EOG,2024-05-11,NaN,,,,,,
3,Investment Research,IR-4,DVN,2024-05-11,NaN,,,,,,
4,Investment Research,IR-5,COP,2024-05-07,NaN,,,,,,


In [ ]:
# Function to find the first empty row of the csv
def find_first_unique_id_with_empty_values(file_path, categories):
    """
    Finds the first unique ID where any of the specified columns have empty values in a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        categories (list of str): List of column names to check for empty values.

    Returns:
        str: The first Unique_ID where any of the specified columns have empty values.
        None: If no such row is found.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over each row to find the first unique ID with any empty values in the specified columns
    for index, row in df.iterrows():
        if row[categories].isnull().any() or (row[categories] == '').any():
            return row['Unique_ID']

    return None  # Return None if no such row is found

# Test function
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
print(unique_id)

IR-1


In [ ]:

def parse_sentiment(text, categories):
    """
    Parses a given text for specified categories and their sentiments.

    Args:
        text (str): The input text containing categories and their sentiments.
        categories (list of str): List of category names to look for in the text.

    Returns:
        dict or str: A dictionary with categories as keys and their corresponding sentiments as values,
                     or "Did not find all categories" if any sentiment is not Positive, Neutral, or Negative.
    """
    results = {}
    valid_sentiments = {"Positive", "Neutral", "Negative"}

    for category in categories:
        pattern = rf"- {re.escape(category)} - (\w+)"
        match = re.search(pattern, text)

        if match:
            sentiment = match.group(1)
            if sentiment not in valid_sentiments:
                return "Did not find all categories"
            results[category] = sentiment
        else:
            return "Did not find all categories"

    return results

def update_csv(file_path, unique_id, sentiment_dict):
    """
    Updates the columns of a CSV file based on the unique ID and sentiment dictionary.

    Args:
        file_path (str): The path to the CSV file.
        unique_id (str): The unique ID of the row to be updated.
        sentiment_dict (dict): A dictionary with categories as keys and their corresponding sentiments as values.

    Returns:
        None
    """
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
    else:
        columns = ['Unique_ID'] + CATEGORIES
        df = pd.DataFrame(columns=columns)

    if unique_id in df['Unique_ID'].values:
        row_index = df[df['Unique_ID'] == unique_id].index
        for category, sentiment in sentiment_dict.items():
            df.loc[row_index, category] = sentiment
    else:
        new_row = {'Unique_ID': unique_id}
        new_row.update(sentiment_dict)
        df = df.append(new_row, ignore_index=True)

    df.to_csv(file_path, index=False)
    print(f"Row with Unique_ID '{unique_id}' has been updated.")

In [ ]:
# Global variables to store model, tokenizer, and llm
model_4bit = None
tokenizer = None
llm = None

# Prompt template for sentiment analysis
template_rev_2 = """<s>Given the text, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

The text is below:
{article_text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
"""

# Define and Load Model and Tokenizer
def initialize_model():
    """Initializes the model and tokenizer."""
    global model_4bit, tokenizer
    if model_4bit is None or tokenizer is None:
        try:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True)
            model_4bit = AutoModelForCausalLM.from_pretrained(
                "meta-llama/Meta-Llama-3-8B-Instruct",
                device_map="auto",
                quantization_config=quantization_config)
            tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
        except Exception as e:
            print(f"Error initializing model or tokenizer: {e}")

# Create Hugging Face Pipeline
def initialize_llm():
    """Initializes the Hugging Face pipeline."""
    global llm
    if llm is None:
        try:
            pipeline_inst = pipeline(
                "text-generation",
                model=model_4bit,
                tokenizer=tokenizer,
                use_cache=True,
                device_map="auto",
                max_length=5000,
                do_sample=True,
                top_k=5,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id)
            llm = HuggingFacePipeline(pipeline=pipeline_inst)
        except Exception as e:
            print(f"Error initializing LLM pipeline: {e}")

# Function to generate response
def generate_response(article_text):
    """Generates the response for the given article text using the LLM."""
    prompt = PromptTemplate(template=template_rev_2, input_variables=["article_text"])
    prompt_str = prompt.format(article_text=article_text)
    response = llm(prompt_str)
    return response[0]['generated_text']

# Function to process the article
def process_article(article_text):
    """Processes the article text and returns the sentiment analysis result."""
    full_response = generate_response(article_text)
    split_response = full_response.split("</s>", 1)
    if len(split_response) > 1:
        final_response = split_response[1]
    else:
        final_response = "Error processing response"
    return final_response

In [ ]:
initialize_model()
initialize_llm()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Import Libraries
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate
import pandas as pd
import torch
import re

# Global variables to store model, tokenizer, and llm
model_4bit = None
tokenizer = None
llm = None

# Prompt template for sentiment analysis
template_rev_2 = """<s>Given the text, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

The text is below:
{article_text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
"""

# Define and Load Model and Tokenizer
def initialize_model():
    """Initializes the model and tokenizer."""
    global model_4bit, tokenizer
    if model_4bit is None or tokenizer is None:
        try:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True)
            model_4bit = AutoModelForCausalLM.from_pretrained(
                "meta-llama/Meta-Llama-3-8B-Instruct",
                device_map="auto",
                quantization_config=quantization_config)
            tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
        except Exception as e:
            print(f"Error initializing model or tokenizer: {e}")

# Create Hugging Face Pipeline
def initialize_llm():
    """Initializes the Hugging Face pipeline."""
    global llm
    if llm is None:
        try:
            pipeline_inst = pipeline(
                "text-generation",
                model=model_4bit,
                tokenizer=tokenizer,
                use_cache=True,
                device_map="auto",
                max_new_tokens=500,  # Set a reasonable number of new tokens to generate
                do_sample=True,
                top_k=5,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id)
            llm = HuggingFacePipeline(pipeline=pipeline_inst)
        except Exception as e:
            print(f"Error initializing LLM pipeline: {e}")

# Function to generate response
def generate_response(article_text):
    """Generates the response for the given article text using the LLM."""
    prompt = PromptTemplate(template=template_rev_2, input_variables=["article_text"])
    prompt_str = prompt.format(article_text=article_text)
    response = llm(prompt_str)
    print(response)  # Debugging line to check the response format
    return response

# Function to process the article
def process_article(article_text):
    """Processes the article text and returns the sentiment analysis result."""
    full_response = generate_response(article_text)
    split_response = full_response.split("</s>", 1)
    if len(split_response) > 1:
        final_response = split_response[1]
    else:
        final_response = "Error processing response"
    return final_response

# Example usage
if __name__ == "__main__":
    initialize_model()
    initialize_llm()
    article_text = "Your article text here..."
    result = process_article(article_text)
    print(result)

# Assuming `text_df` is your dataframe containing articles
for index, row in text_df.iterrows():
    unique_id = row['Unique_ID']
    article_text = row['Article Text']

    full_response = process_article(article_text)  # Pass article_text as a positional argument
    sentiment_dict = parse_sentiment(full_response, CATEGORIES)

    if sentiment_dict != "Did not find all categories":
        update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)
    else:
        print(f"Failed to parse sentiments for Unique_ID '{unique_id}'")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s>Given the text, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

The text is below:
Your article text here...

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
DO NOT include any additional

KeyboardInterrupt: 